# TensorFlow Assignment: Multilayer Perceptron (MLP) and Convolutional Neural Network (CNN)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: [Weiguo Ye]

Now that you've run through a simple logistic regression model on MNIST, let's see if we can do better (Hint: we can). For this assignment, you'll build a multilayer perceptron (MLP) and a convolutional neural network (CNN), two popular types of neural networks, and compare their performance. Some potentially useful code:

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Import data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Helper functions for creating weight variables
def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Tensorflow Functions that might also be of interest:
# tf.nn.sigmoid()
# tf.nn.relu()

### Multilayer Perceptron

Build a multilayer perceptron for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> fully connected (500 hidden units) -> nonlinearity (Sigmoid/ReLU) -> fully connected (10 hidden units) -> softmax

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

In [8]:
# Model Inputs
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# Define the graph


### Create your MLP here##
### Make sure to name your MLP output as y_mlp ###
W1 = tf.Variable(tf.truncated_normal(shape=[784, 500], stddev=0.1))
b1 = tf.Variable(tf.constant(0.1, shape=[500]))
W2 = tf.Variable(tf.truncated_normal(shape=[500, 10], stddev=0.1))
b2 = tf.Variable(tf.constant(0.1, shape=[10]))

u0 = tf.matmul(x, W1) + b1
y0 = tf.nn.sigmoid(u0)
# y0 = tf.nn.relu(u0)
y_mlp = tf.matmul(y0, W2) + b2

# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_mlp))

# Optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_mlp, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
#    with tf.device("/cpu:0"):
    # Initialize all variables
    sess.run(tf.global_variables_initializer())

    # Training regimen
    for i in range(4000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(100)
                validation_accuracy += (1/10.0) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))

        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.09
step 250, validation accuracy 0.887
step 500, validation accuracy 0.907
step 750, validation accuracy 0.912
step 1000, validation accuracy 0.924
step 1250, validation accuracy 0.928
step 1500, validation accuracy 0.938
step 1750, validation accuracy 0.958
step 2000, validation accuracy 0.956
step 2250, validation accuracy 0.954
step 2500, validation accuracy 0.944
step 2750, validation accuracy 0.967
step 3000, validation accuracy 0.967
step 3250, validation accuracy 0.957
step 3500, validation accuracy 0.964
step 3750, validation accuracy 0.97
test accuracy 0.9633


#### Comparison

How do the sigmoid and rectified linear unit (ReLU) compare?


***

Firstly, relu converge much faster then sigmoid. The main reason is that the gradient directly flows through relu layer without discount, while sigmoid can diminish the gradient.

Secondly, 

* test result for sigmoid:

step 0, validation accuracy 0.09
step 250, validation accuracy 0.887
step 500, validation accuracy 0.907
step 750, validation accuracy 0.912
step 1000, validation accuracy 0.924
step 1250, validation accuracy 0.928
step 1500, validation accuracy 0.938
step 1750, validation accuracy 0.958
step 2000, validation accuracy 0.956
step 2250, validation accuracy 0.954
step 2500, validation accuracy 0.944
step 2750, validation accuracy 0.967
step 3000, validation accuracy 0.967
step 3250, validation accuracy 0.957
step 3500, validation accuracy 0.964
step 3750, validation accuracy 0.97
test accuracy 0.9633


* test result for relu:

step 0, validation accuracy 0.046
step 250, validation accuracy 0.956
step 500, validation accuracy 0.939
step 750, validation accuracy 0.961
step 1000, validation accuracy 0.965
step 1250, validation accuracy 0.965
step 1500, validation accuracy 0.97
step 1750, validation accuracy 0.968
step 2000, validation accuracy 0.982
step 2250, validation accuracy 0.976
step 2500, validation accuracy 0.976
step 2750, validation accuracy 0.982
step 3000, validation accuracy 0.976
step 3250, validation accuracy 0.979
step 3500, validation accuracy 0.986
step 3750, validation accuracy 0.975
test accuracy 0.9787



***

### Convolutional Neural Network

Build a simple 2-layer CNN for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> CNN (32 5x5 filters) -> nonlinearity (ReLU) ->  (2x2 max pool) -> CNN (64 5x5 filters) -> nonlinearity (ReLU) -> (2x2 max pool) -> fully connected (1024 hidden units) -> nonlinearity (ReLU) -> fully connected (10 hidden units) -> softmax

Some additional functions that you might find helpful:

In [4]:
# Convolutional neural network functions
def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Tensorflow Function that might also be of interest:
# tf.reshape()

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

*Hint: Convolutional Neural Networks are spatial models. You'll want to transform the flattened MNIST vectors into images for the CNN. Similarly, you might want to flatten it again sometime before you do a softmax. You also might want to look into the  [conv2d() documentation](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d) to see what shape kernel/filter it's expecting.*

In [6]:
# Model Inputs
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# Define the graph
Conv0 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
Conv1 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
Fc0_W = tf.Variable(tf.truncated_normal([64*49, 1024], stddev=0.1))
Fc0_b = tf.Variable(tf.constant(0.1, shape=[1024]))
Fc1_W = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
Fc1_b = tf.Variable(tf.constant(0.1, shape=[10]))

### Create your CNN here##
### Make sure to name your CNN output as y_conv ###
x_ = tf.reshape(x, [-1, 28, 28, 1])
conv0 = conv2d(x_, Conv0)
relu0 = tf.nn.relu(conv0)
pool0 = max_pool_2x2(relu0)
conv1 = conv2d(pool0, Conv1)
relu1 = tf.nn.relu(conv1)
pool1 = max_pool_2x2(relu1)
flat = tf.reshape(pool1, [-1, 64*49])
fc0 = tf.matmul(flat, Fc0_W) + Fc0_b
relu2 = tf.nn.relu(fc0)
y_conv = tf.matmul(relu2, Fc1_W) + Fc1_b


# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# Optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


with tf.Session() as sess:
    
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    # Training regimen
    for i in range(10000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(50)
                validation_accuracy += (1/10.0) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})
        
    test_accuracy = 0.0
    for v in range(10):
        batch = mnist.test.next_batch(50)
        test_accuracy += (1/10.0) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
    print('test accuracy %g' % test_accuracy)

step 0, validation accuracy 0.074
step 250, validation accuracy 0.954
step 500, validation accuracy 0.968
step 750, validation accuracy 0.972
step 1000, validation accuracy 0.964
step 1250, validation accuracy 0.984
step 1500, validation accuracy 0.972
step 1750, validation accuracy 0.97
step 2000, validation accuracy 0.984
step 2250, validation accuracy 0.994
step 2500, validation accuracy 0.978
step 2750, validation accuracy 0.976
step 3000, validation accuracy 0.978
step 3250, validation accuracy 0.988
step 3500, validation accuracy 0.982
step 3750, validation accuracy 0.988
step 4000, validation accuracy 0.982
step 4250, validation accuracy 0.994
step 4500, validation accuracy 0.994
step 4750, validation accuracy 0.994
step 5000, validation accuracy 0.984
step 5250, validation accuracy 0.996
step 5500, validation accuracy 0.992
step 5750, validation accuracy 0.99
step 6000, validation accuracy 0.986
step 6250, validation accuracy 0.996
step 6500, validation accuracy 0.99
step 6750,

Some differences from the logistic regression model to note:

- The CNN model might take a while to train. Depending on your machine, you might expect this to take up to half an hour. If you see your validation performance start to plateau, you can kill the training.

- The logistic regression model we used previously was pretty basic, and as such, we were able to get away with using the GradientDescentOptimizer, which performs implements the gradient descent algorithm. For more difficult optimization spaces (such as the ones deep networks pose), we might want to use more sophisticated algorithms. Prof David Carlson has a lecture on this later.
    
- Because of the larger size of our network, notice that our minibatch size has shrunk.
    
- We've added a validation step every 250 minibatches. This let's us see how our model is doing during the training process, rather than sit around twiddling our thumbs and hoping for the best when training finishes. This becomes especially significant as training regimens start approaching days and weeks in length. Normally, we validate on the entire validation set, but for the sake of time we'll just stick to 10 validation minibatches (500 images) for this homework assignment.

#### Comparison

How do the MLP and CNN compare in accuracy? Training time? Why would you use one vs the other? Is there a problem you see with MLPs when applied to other image datasets?

***

CNN outperforms MLP in terms of accuracy.

Previously crafted MLP spends less time in training than this CNN. Theoretically, the forward step of a CNN should not spend more time than a MLP with the same number of layers. The number of operations for the forward pass through a convolutoin layer is O(N\*filter_w\*filter_h\*filter_num), and for fully connected layer O(N\*M), and normally filter_w\*filter_h\*filter_num will not be larger than M. However, this CNN has much more layer than previous MLP.

In practice, when dealing with image data, I will use CNN because: (1) CNN captures the regional relation of the objects in the image, while flattening the image afface this information, (2) CNN provides translation invariance will the pooling layers, (3) convolution layer has less parameters (filter_w\*filter_h\*filter_num) than fully connected layer (N\*M).

MLPs have much more parameters than CNNs, so they easily get overfitted. Therefore MLPs can perform badly when applied to other image datasets.



* Test Result:

step 0, validation accuracy 0.074
step 250, validation accuracy 0.954
step 500, validation accuracy 0.968
step 750, validation accuracy 0.972
step 1000, validation accuracy 0.964
step 1250, validation accuracy 0.984
step 1500, validation accuracy 0.972
step 1750, validation accuracy 0.97
step 2000, validation accuracy 0.984
step 2250, validation accuracy 0.994
step 2500, validation accuracy 0.978
step 2750, validation accuracy 0.976
step 3000, validation accuracy 0.978
step 3250, validation accuracy 0.988
step 3500, validation accuracy 0.982
step 3750, validation accuracy 0.988
step 4000, validation accuracy 0.982
step 4250, validation accuracy 0.994
step 4500, validation accuracy 0.994
step 4750, validation accuracy 0.994
step 5000, validation accuracy 0.984
step 5250, validation accuracy 0.996
step 5500, validation accuracy 0.992
step 5750, validation accuracy 0.99
step 6000, validation accuracy 0.986
step 6250, validation accuracy 0.996
step 6500, validation accuracy 0.99
step 6750, validation accuracy 0.992
step 7000, validation accuracy 0.988
step 7250, validation accuracy 0.984
step 7500, validation accuracy 0.992
step 7750, validation accuracy 0.99
step 8000, validation accuracy 0.986
step 8250, validation accuracy 0.996
step 8500, validation accuracy 0.988
step 8750, validation accuracy 0.99
step 9000, validation accuracy 0.994
step 9250, validation accuracy 0.994
step 9500, validation accuracy 0.986
step 9750, validation accuracy 0.984
test accuracy 0.988


***